In [9]:
donusturulecek_liste

,0
0,"2.21,3.10,2.79,1.74,1.82,1.67,1.91,1.29,1.23,1..."
1,"1.77,3.25,3.83,1.61,2.00,1.89,1.68,1.16,1.22,1..."
2,"2.53,2.88,2.55,1.49,2.22,1.91,1.66,1.34,1.27,1..."
3,"3.01,2.79,2.26,1.42,2.39,2.03,1.59,1.43,1.29,1..."
4,"1.52,3.51,5.25,1.53,2.12,2.18,1.51,1.08,1.18,1..."
...,...
882,"1.83,3.35,3.32,1.94,1.63,1.52,2.00,1.19,1.19,1..."
883,"1.53,3.66,4.62,1.85,1.68,1.65,1.83,1.09,1.15,1..."
884,"1.72,3.19,3.99,1.59,1.99,1.77,1.70,1.13,1.21,1..."
885,"5.41,3.89,1.32,1.86,1.53,1.65,1.65,2.10,1.06,0..."


In [1]:
import pandas as pd
import selenium
from selenium import webdriver


path = "D:\chromedriver\chromedriver.exe"

driver = webdriver.Chrome(path)
urls = []
link_devami = "basbasa"
# "linkler.txt" dosyasını okuma
mac_sayaci = 0
donusturulecek_liste = []
with open("D:/links.txt", "r") as dosya:
    # Dosyadaki her satırı okuma
    for satir in dosya:
        # Satır sonundaki newline karakterini ve gereksiz boşlukları temizleme
        url = satir.strip()
        # URL'i urls listesine ekleme
        urls.append(url + link_devami)

for x in urls:
    try:
        driver.get(x)
        elements = driver.find_element("xpath", "//*[@id='middle']/div[4]/div[2]")
        new = elements.text
        df = pd.DataFrame(new.split("\n"))
        df2 = df.iloc[10:-11:1]
        df2 = df2.drop([11, 12, 14, 15, 17, 18, 20, 21, 23, 24, 26])
        df2 = df2.reset_index(drop=True)
        df2 = df2.rename(columns={0: "values"})
        df2 = df2.fillna('0')
        df2 = df2.apply(lambda x: x.str.replace(' ', ','))
        df2 = df2.apply(lambda x: x.str.replace('-', '0'))

        result = ','.join([','.join(row.split(',')) for row in df2.values.flatten()])
        result = result.rstrip(",")

        halftime = driver.find_element("xpath", "//*[@id='matchdetailhtscore']")
        ht = halftime.text
        ht_new = ht.replace("IY ", "").replace("-", ",")
        ev, dep = ht_new.split(",")
        ev = int(ev)
        dep = int(dep)
        if (ev > dep):
            iy_sonuc = 1
        if (ev < dep):
            iy_sonuc = 2
        if (ev == dep):
            iy_sonuc = 0

        fulltime = driver.find_element("xpath", "//*[@id='matchdetailscore']")
        ft = fulltime.text
        ft_new = ft.replace("-", ",")
        home, away = ft_new.split(",")
        home = int(home)
        away = int(away)
        if (home > away):
            ft_sonuc = 1
        if (home < away):
            ft_sonuc = 2
        if (home == away):
            ft_sonuc = 0

        detay_ev = driver.find_element("xpath", "//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table")
        detay2_ev = detay_ev.text

        ev_veriler = pd.DataFrame(detay2_ev.split("\n"))
        ev_veriler2 = ev_veriler.iloc[2::3]
        ev_veriler2 = ev_veriler2.reset_index(drop=True)
        ev_veriler2 = ev_veriler2.rename(columns={0: "values"})
        ev_veriler2 = ev_veriler2.fillna('0')
        ev_veriler2 = ev_veriler2.apply(lambda x: x.str.replace('%', ''))
        ev_sonuc = ','.join([','.join(row.split(',')) for row in ev_veriler2.values.flatten()])

        detay_dep = driver.find_element("xpath", "//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[2]")
        detay2_dep = detay_dep.text

        dep_veriler = pd.DataFrame(detay2_dep.split("\n"))
        dep_veriler2 = dep_veriler.iloc[4::3]
        dep_veriler2 = dep_veriler2.reset_index(drop=True)
        dep_veriler2 = dep_veriler2.rename(columns={0: "values"})
        dep_veriler2 = dep_veriler2.fillna('0')
        dep_veriler2 = dep_veriler2.apply(lambda x: x.str.replace('%', ''))
        dep_sonuc = ','.join([','.join(row.split(',')) for row in dep_veriler2.values.flatten()])

        lig = driver.find_element_by_xpath("//*[@id='middle']/div[1]/div/header/h3/a/span[3]")
        lig = lig.text

        if (lig == "Türkiye Süper Lig"):
            lig_id = 1500
        if (lig == "Fransa 2.Lig"):
            lig_id = 1501
        if (lig == "Fransa 1.Lig"):
            lig_id = 1502
        if (lig == "Hollanda Eredivisie"):
            lig_id = 1503
        if (lig == "İngiltere Premier Lig"):
            lig_id = 1504
        if (lig == "İngiltere Championship"):
            lig_id = 1505
        if (lig == "İspanya La Liga"):
            lig_id = 1506
        if (lig == "İtalya Serie A"):
            lig_id = 1507
        if (lig == "Almanya Bundesliga I"):
            lig_id = 1508

        donusturulecek_liste.append(
            result + "," + ev_sonuc + "," + dep_sonuc + "," + ht_new + "," + ft_new + "," + str(iy_sonuc) + "," + str(
                ft_sonuc) + "," + str(lig_id))
        mac_sayaci = mac_sayaci + 1
        print(result + "," + ev_sonuc + "," + dep_sonuc + "," + ht_new + "," + ft_new + "," + str(iy_sonuc) + "," + str(
            ft_sonuc) + "," + str(lig_id))
    except Exception as hata:
        print(str(hata))
        continue

donusturulecek_liste = pd.DataFrame(donusturulecek_liste)
donusturulecek_liste.to_csv("D:/YENI_CEKILEN_VERILER2.csv", index=False)
print("------------------------------------------------")
print("------------------------------------------------")
print(mac_sayaci)

1.63,3.69,4.21,2.01,1.64,1.64,1.98,1.13,1.17,1.86,1.12,1.46,1.45,1.39,2.48,3.94,1.16,1.36,2.67,0,0,0,3.94,1.92,3.50,12.50,2.11,2.13,4.39,1.94,2.66,4.15,1.80,1.76,2.45,16.00,49.00,4.51,5.91,9.98,25.00,16.65,7.38,25,75,0,25,75,0,2,0,5,1,1,1,1504
2.35,3.18,2.62,1.84,1.76,1.63,2.02,1.33,1.22,1.40,1.22,1.52,1.26,1.35,2.65,3.59,1.20,1.30,2.93,0,0,0,3.56,1.90,3.78,15.00,2.87,2.01,3.11,2.64,2.47,2.86,1.84,1.73,3.84,14.20,32.00,5.89,5.00,6.44,31.00,14.30,4.35,66,33,0,25,50,25,0,1,0,1,2,2,1504
1.93,3.32,3.32,1.83,1.77,1.67,1.95,1.21,1.21,1.60,1.16,1.51,1.35,1.34,2.66,3.54,1.21,1.29,2.97,0,0,0,3.51,1.89,3.83,15.45,2.47,2.02,3.76,2.24,2.50,3.47,1.82,1.74,3.04,14.90,40.50,4.98,5.16,7.82,27.50,15.40,5.75,25,25,50,66,33,0,1,1,2,1,0,1,1504
1.60,3.69,4.45,1.98,1.65,1.69,1.92,1.11,1.17,1.91,1.10,1.46,1.48,1.38,2.49,3.91,1.17,1.35,2.69,0,0,0,3.90,1.92,3.53,12.65,2.09,2.14,4.59,1.90,2.65,4.35,1.81,1.75,2.38,16.30,53.00,4.42,5.84,10.45,25.50,17.05,7.90,25,25,50,50,25,25,1,1,3,1,0,1,1504
2.35,3.48,2.45,2.43

1.51,3.76,5.17,1.81,1.79,1.87,1.74,1.08,1.16,2.04,1.08,1.48,1.51,1.34,2.67,3.48,1.21,1.29,3.02,0,0,0,3.45,1.89,3.89,16.00,2.01,2.09,5.25,1.82,2.60,4.97,1.80,1.76,2.25,17.80,65.00,4.12,5.66,11.60,27.00,18.80,9.44,40,20,40,40,40,20,1,0,2,0,1,1,1504
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

4.46,3.47,1.62,1.72,1.85,1.85,1.72,1.84,1.18,1.10,1.41,1.49,1.08,1.29,2.77,3.21,1.24,1.24,3.18,0,0,0,3.17,1.87,4.11,18.35,4.76,2.00,2.15,4.44,2.47,1.94,1.80,1.74,8.14,17.60,27.50,9.98,5.16,4.23,58.00,16.75,2.46,33,33,33,33,33,33,1,0,2,3,1,2,1504
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

2.82,3.09,2.20,1.73,1.84,1.69,1.89,1.43,1.22,1.27,1.26,1.51,1.18,1.30,2.77,3.25,1.23,1.25,3.14,0,0,0,3.21,1.86,4.05,17.85,3.34,1.94,2.75,

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

8.71,5.76,1.18,2.78,1.31,1.64,1.86,2.99,0,0,1.88,1.26,0,1.65,1.91,5.80,1.05,1.70,1.87,0,0,0,6.64,2.21,2.67,5.75,6.90,2.66,1.52,7.45,3.41,1.39,1.76,1.77,15.55,24.15,22.80,22.35,10.05,3.97,105.00,25.00,1.58,42,28,28,16,33,50,0,1,1,3,2,2,1504
1.89,3.32,3.31,1.83,1.74,1.61,1.93,1.20,1.20,1.59,1.15,1.48,1.34,1.35,2.56,3.50,1.20,1.29,2.88,0,0,0,3.62,1.89,3.66,14.10,2.41,2.03,3.71,2.20,2.51,3.44,1.80,1.72,2.96,14.70,40.00,4.93,5.23,7.88,27.00,15.15,5.72,37,12,50,37,50,12,0,2,0,2,2,2,1504
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}


Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

Message: no such element: Unable to locate element: {"method":"xpath","sele

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

1.58,3.44,4.80,1.62,1.99,1.92,1.61,1.09,1.18,1.88,1.08,1.50,1.42,1.26,2.91,2.91,1.29,1.19,3.55,0,0,0,2.94,1.86,4.42,21.50,2.12,1.96,5.10,1.92,2.42,4.74,1.80,1.73,2.11,14.15,36.00,3.47,4.18,8.60,23.70,14.95,7.32,36,18,45,63,27,9,0,0,4,0,0,1,1504
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

Message: no suc

...

1.25,5.00,7.78,2.26,1.47,1.75,1.76,0,1.07,2.69,0,1.34,1.79,1.54,2.06,4.53,1.11,1.47,2.26,0,0,0,4.89,2.00,3.02,8.56,1.54,2.56,7.25,1.49,3.07,6.83,1.77,1.77,1.66,23.70,98.00,3.96,8.39,19.20,25.00,24.15,14.80,83,16,0,80,0,20,0,0,1,0,0,1,1500
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

2.67,3.31,2.19,2.22,1.49,1.37,2.43,1.44,1.20,1.30,1.29,1.41,1.22,1.48,2.18,4.55,1.11,1.47,2.25,0,0,0,4.93,2.01,3.01,8.47,3.00,2.19,2.59,2.83,2.64,2.43,1.83,1.70,4.32,13.25,26.50,7.08,5.68,6.02,30.00,13.10,3.44,50,33,16,14,42,42,0,0,0,0,0,0,1500
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

1.91,3.20,3.38,1.73,1.84,1.62,1.91,1.19,1.21,1.58,1.14,1.50,1.33,1.33,2.69,3.23,1.23,1.25,3.13,0,0,0,3.31,1.87,3.81,16.60,2.44,1.98,3.81,2.23,2.43

2.65,3.13,2.30,1.73,1.83,1.63,1.89,1.40,1.22,1.30,1.24,1.51,1.20,1.31,2.70,3.23,1.23,1.25,3.13,0,0,0,3.32,1.87,3.93,16.65,3.17,1.96,2.84,2.90,2.41,2.60,1.81,1.71,4.46,14.45,31.50,6.39,4.83,5.68,34.50,14.30,3.78,28,42,28,66,0,33,0,0,0,0,0,0,1500
1.63,3.83,3.90,2.49,1.39,1.40,2.34,1.14,1.14,1.82,1.13,1.36,1.48,1.56,2.02,5.20,1.08,1.60,2.02,0,0,0,5.78,2.11,2.80,6.80,2.03,2.33,3.93,1.89,2.84,3.81,1.80,1.73,2.37,14.80,40.50,4.87,6.76,10.15,22.15,15.20,6.57,62,25,12,42,28,28,2,0,2,1,1,1,1500
2.38,3.31,2.44,2.16,1.52,1.42,2.30,1.36,1.20,1.37,1.25,1.42,1.25,1.46,2.23,4.40,1.12,1.45,2.32,0,0,0,4.73,1.99,3.08,8.99,2.78,2.16,2.82,2.60,2.62,2.65,1.82,1.70,3.79,13.35,28.50,6.35,5.64,6.48,28.00,13.40,3.90,37,25,37,12,50,37,2,0,3,3,1,0,1500
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

1.97,3.30,3.13,1.90,1.68,1.55,2.00,1.22,1.20,1.54,1.17,1.46,1.32,1.37,2.48,3.68

1.96,3.34,3.11,2.01,1.60,1.49,2.12,1.22,1.20,1.55,1.17,1.44,1.34,1.41,2.35,4.00,1.14,1.38,2.52,0,0,0,4.22,1.94,3.28,10.75,2.42,2.11,3.45,2.24,2.58,3.23,1.82,1.71,2.64,11.45,29.00,4.47,4.64,6.48,21.30,11.70,4.43,50,40,10,60,10,30,0,2,0,3,2,2,1500
3.61,3.65,1.72,2.49,1.39,1.37,2.41,1.72,1.16,1.17,1.44,1.37,1.15,1.56,2.02,5.23,1.08,1.60,2.02,0,0,0,5.82,2.11,2.80,6.77,3.69,2.32,2.11,3.55,2.80,1.98,1.82,1.71,5.03,11.75,18.55,7.86,5.41,4.31,30.50,11.50,2.22,33,33,33,22,33,44,0,1,1,1,2,0,1500
2.06,3.31,2.91,1.96,1.63,1.51,2.08,1.25,1.20,1.50,1.19,1.45,1.31,1.39,2.41,3.84,1.16,1.35,2.63,0,0,0,4.02,1.92,3.38,11.65,2.54,2.08,3.31,2.34,2.55,3.08,1.81,1.72,2.81,11.60,28.00,4.57,4.56,6.05,22.15,11.80,4.14,22,44,33,70,10,20,1,1,4,1,0,1,1500
2.01,3.39,2.94,2.13,1.53,1.45,2.22,1.24,1.19,1.52,1.19,1.42,1.33,1.45,2.25,4.29,1.12,1.43,2.36,0,0,0,4.58,1.98,3.12,9.40,2.46,2.15,3.28,2.28,2.63,3.08,1.81,1.71,2.71,11.35,27.50,4.63,4.82,6.28,20.95,11.60,4.13,30,20,50,36,45,18,3,1,5,1,1,1,1500
1.14,6.29,9.88,2.9

3.78,3.47,1.73,1.89,1.68,1.61,1.92,1.72,1.18,1.15,1.39,1.45,1.12,1.37,2.49,3.65,1.18,1.31,2.75,0,0,0,3.79,1.90,3.53,12.85,4.07,2.09,2.23,3.81,2.56,2.03,1.73,1.66,5.53,12.95,20.95,7.44,4.63,3.94,36.00,12.45,2.31,25,41,33,8,16,75,0,1,2,1,2,1,1500
1.82,3.53,3.34,2.28,1.46,1.42,2.30,1.20,1.18,1.64,1.16,1.39,1.38,1.50,2.14,4.69,1.10,1.50,2.19,0,0,0,5.10,2.03,2.96,8.06,2.24,2.23,3.54,2.09,2.71,3.38,1.74,1.64,2.40,11.45,29.50,4.38,5.11,7.14,19.45,11.75,4.69,41,16,41,45,27,27,1,1,1,1,0,0,1500
1.83,3.52,3.30,2.19,1.50,1.45,2.22,1.20,1.18,1.63,1.16,1.40,1.37,1.47,2.21,4.45,1.12,1.46,2.29,0,0,0,4.79,2.00,3.05,8.82,2.28,2.20,3.55,2.11,2.67,3.37,1.74,1.65,2.43,11.70,29.50,4.33,5.02,6.96,19.80,12.00,4.66,66,33,0,63,18,18,2,0,2,0,1,1,1500
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

2.57,2.92,2.49,1.59,2.03,1.73,1.78,1.35,1.25,1.32,1.22,1.55,1.21,1.26,2.95,2.88,1

4.61,3.88,1.52,2.36,1.44,1.47,2.15,1.97,1.14,1.09,1.53,1.36,1.10,1.52,2.09,4.88,1.09,1.53,2.13,0,0,0,5.34,2.06,2.90,7.55,4.44,2.33,1.92,4.32,2.83,1.79,1.73,1.65,7.55,15.20,21.45,11.05,6.36,4.39,36.00,14.70,2.10,14,28,57,21,35,42,0,1,0,2,2,2,1500
2.20,3.30,2.67,2.00,1.61,1.48,2.14,1.30,1.21,1.44,1.21,1.45,1.28,1.40,2.37,3.96,1.15,1.37,2.54,0,0,0,4.18,1.93,3.30,10.90,2.66,2.09,3.09,2.46,2.56,2.87,1.74,1.64,3.37,13.10,30.00,5.51,5.20,6.48,26.50,13.25,4.22,42,21,35,21,42,35,1,2,1,2,2,2,1500
1.38,4.63,5.20,2.99,1.28,1.39,2.38,1.07,1.09,2.24,1.08,1.29,1.65,1.70,1.83,6.35,0,1.80,1.76,0,0,0,7.47,2.31,2.59,5.05,1.75,2.55,4.70,1.62,3.16,4.79,1.70,1.67,1.85,16.65,36.00,4.28,8.10,13.10,19.05,16.70,8.32,71,28,0,50,7,42,4,3,5,3,1,1,1500
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='middle']/div[3]/div[1]/div/div[3]/div/div[1]/table"}
  (Session info: chrome=113.0.5672.127)

'[11, 12, 14, 15, 17, 18, 20, 21, 23, 24, 26] not found in axis'
'[11, 12, 14, 15,

In [16]:
donusturulecek_liste

['2.21,3.10,2.79,1.74,1.82,1.67,1.91,1.29,1.23,1.45,1.20,1.54,1.27,1.31,2.78,3.31,1.22,1.24,3.11,0,0,0,3.27,1.86,3.99,17.20,2.79,1.97,3.34,2.54,2.42,3.06,1.84,1.73,3.63,14.45,36.50,5.54,4.82,6.73,31.00,14.70,4.79,33,33,33,25,75,0,1,0,2,1,1,1,1501',
 '1.77,3.25,3.83,1.61,2.00,1.89,1.68,1.16,1.22,1.70,1.11,1.57,1.37,1.26,3.04,2.92,1.28,1.19,3.60,0,0,0,2.86,1.86,4.56,23.25,2.39,1.92,4.40,2.14,2.39,4.02,1.81,1.75,2.83,16.45,53.00,4.45,4.77,8.54,30.00,17.20,7.02,75,25,0,50,25,25,0,0,3,0,0,1,1501',
 '2.53,2.88,2.55,1.49,2.22,1.91,1.66,1.34,1.27,1.35,1.21,1.65,1.21,1.22,3.33,2.62,1.35,1.13,4.12,0,0,0,2.56,1.87,5.22,30.50,3.24,1.82,3.25,2.91,2.23,2.93,1.85,1.72,4.39,15.35,39.50,5.85,4.13,5.89,39.50,15.35,4.43,50,0,50,33,33,33,1,0,1,0,1,1,1501',
 '3.01,2.79,2.26,1.42,2.39,2.03,1.59,1.43,1.29,1.25,1.23,1.68,1.16,1.19,3.55,2.43,1.40,1.10,4.59,0,0,0,2.36,1.89,5.81,38.00,3.75,1.78,2.98,3.36,2.17,2.67,1.86,1.71,5.40,15.90,39.00,6.65,3.88,5.24,47.50,15.55,3.85,50,25,25,25,50,25,0,0,1,0,0,1,1501',
 '1